In [1]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
! pip install transformers==4.27.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 11.0 MB/s eta 0:00:00


In [4]:
! pip install torchtext==0.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.


In [5]:
import torch

In [6]:
device = torch.device("cuda")
torch.cuda.init()

In [7]:
torch.cuda.empty_cache()


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [11]:
config = GPT2Config(vocab_size=50257, n_positions=512, n_ctx=512, n_embd=512, n_layer=12, n_head=8)

In [12]:
# model = GPT2LMHeadModel(config)
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [13]:
train_data = TextDataset(tokenizer=tokenizer, file_path="/content/gdrive/My Drive/byupccl/byupccl_train.txt", block_size=256)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [14]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [15]:
training_args = TrainingArguments(
    output_dir='/content/gdrive/My Drive/byupccl/',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    data_collator=data_collator,
)

In [17]:
len(train_data.examples)

15820

In [18]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.096500
1000,3.000700
1500,2.968700


TrainOutput(global_step=1978, training_loss=3.007257629332817, metrics={'train_runtime': 1377.3933, 'train_samples_per_second': 11.485, 'train_steps_per_second': 1.436, 'total_flos': 2066819973120000.0, 'train_loss': 3.007257629332817, 'epoch': 1.0})

In [ ]:
model = 

In [19]:
def getResponse(input_text, model,tokenizer, device):
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  input_ids = input_ids.to(device)
  attention_mask = torch.LongTensor([1] * len(input_ids))
  output_ids = model.generate(input_ids,pad_token_id=tokenizer.eos_token_id, max_length=50, num_beams=5, 
                                      num_return_sequences=3, 
                                      no_repeat_ngram_size=2, 
                                      early_stopping=True)
  output_text1 = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  output_text2 = tokenizer.decode(output_ids[1], skip_special_tokens=True)
  output_text3 = tokenizer.decode(output_ids[2], skip_special_tokens=True)
  # parts = output_text.split("\n")
  # bot_response = parts[-2][4:] 
  # bot_response.strip()
  return output_text1, output_text2, output_text3

In [31]:
import re
input_text = "User: Hey, how are you doing?"
input_text = input_text
output_text1, output_text2, output_text3 = getResponse(input_text, model, tokenizer, device)

bot_response = re.search(r'Bot: (.+)', output_text1).group(1)
print(bot_response)

I am doing great! I'm doing pretty good. I've been working on it for a while now, but I feel like it's going to be a long time.
